In [7]:
import numpy as np
import pandas as pd
from vtk import *
from vtk.util import numpy_support
import os

from tqdm import tqdm

In [8]:
CHASTE_FILE = './chaste/'

if not os.path.exists(CHASTE_FILE):
    os.makedirs(CHASTE_FILE)
    
FILE_VTK = './MESH1.vtu'
CHASTE_NODE = './chaste/data.node'
CHASTE_ELE = './chaste/data.ele'
CHASTE_FACE = './chaste/data.face'
CHASTE_AXI = './chaste/data.axi'
CHASTE_CSV = './chaste/data.csv'

In [9]:
reader = vtkXMLUnstructuredGridReader()
reader.SetFileName(FILE_VTK)
reader.Update()
assert(os.path.exists(FILE_VTK))

In [10]:
data = reader.GetOutput()
points = data.GetPoints()
cells = data.GetCells()
print data, points, cells

vtkUnstructuredGrid (0x1c53b60)
  Debug: Off
  Modified Time: 598
  Reference Count: 2
  Registered Events: (none)
  Information: 0x1a9bfa0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 626
  Field Data:
    Debug: Off
    Modified Time: 553
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 1718676
  Number Of Cells: 9976600
  Cell Data:
    Debug: Off
    Modified Time: 561
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
    Scalars: (none)
    Vectors: (none)
    Normals: (none)
    TCoords: (none)
    Tensors: (none)
    GlobalIds: (none)
    PedigreeIds: (none)
    EdgeFlag: (none)
  Point Data:
    Debug: Off
    Modified Time: 598
    Reference Count: 1
    R

In [11]:
points_array = numpy_support.vtk_to_numpy(points.GetData())
cells_array = numpy_support.vtk_to_numpy(cells.GetData())
scalars_array = numpy_support.vtk_to_numpy(reader.GetOutput().GetPointData().GetAbstractArray('Color'))

print points_array
print cells_array
print scalars_array

[[ 2.42452955  1.53688562  4.04003382]
 [ 2.41604114  1.53202176  4.07285881]
 [ 2.39152527  1.54640925  4.07285881]
 ..., 
 [ 1.83399839  2.47370691  3.55495606]
 [ 3.10052833  2.28543999  4.33637869]
 [ 2.97804635  4.2595794   2.92013732]]
[      3      10   81577 ...,  595514   72658 1193141]
[ 0.  0.  0. ...,  0.  0.  0.]


In [12]:
max(scalars_array)

1.0

In [13]:
vectors = numpy_support.vtk_to_numpy(data.GetPointData().GetAbstractArray('Fibers'))
vectors

array([[-0.90615493,  0.247123  ,  0.34323969],
       [-0.90615493,  0.247123  ,  0.34323969],
       [-0.88484681,  0.24654818,  0.39529753],
       ..., 
       [-0.74907786,  0.27869657,  0.601008  ],
       [ 0.25457823, -0.05401361,  0.96554255],
       [-0.29722399,  0.37234417,  0.87921429]], dtype=float32)

In [14]:
with open(CHASTE_NODE, 'w') as fl:
    fl.write("{count} 3 0 0\n".format(count = len(points_array)))
    for i in tqdm(xrange(len(points_array))):
        fl.write("{n} {x} {y} {z}\n".format(n=i, x=points_array[i][0], 
                                            y=points_array[i][1], 
                                            z=points_array[i][2]))

100%|██████████| 1718676/1718676 [00:05<00:00, 326849.24it/s]


In [15]:
j = 0
lst = []

while j < len(cells_array):
    if cells_array[j] == 4:
        lst.append([cells_array[j+1], cells_array[j+2], cells_array[j+3], cells_array[j+4]])
    j += cells_array[j] + 1

In [16]:
count_torso = 0

with open(CHASTE_ELE, 'w') as fl:
    fl.write("{count} 4 1\n".format(count = len(lst)))
    for i in tqdm(xrange(len(lst))):
        fl.write("{n} {p1} {p2} {p3} {p4} {is_torso}\n".format(n=i, 
                                                      p1=lst[i][0], 
                                                      p2=lst[i][1],  
                                                      p3=lst[i][2], 
                                                      p4=lst[i][3],
                                                      is_torso=0))

100%|██████████| 9323944/9323944 [00:26<00:00, 353513.10it/s]


In [17]:
with open(CHASTE_AXI, 'w') as fl:
    fl.write("{count}\n".format(count = len(lst)))
    for i in tqdm(xrange(len(lst))):
        vec1 = np.zeros(3) if np.linalg.norm(vectors[lst[i][0]]) < 0.000001 else vectors[lst[i][0]] / np.linalg.norm(vectors[lst[i][0]])
        vec2 = np.zeros(3) if np.linalg.norm(vectors[lst[i][1]]) < 0.000001 else vectors[lst[i][1]] / np.linalg.norm(vectors[lst[i][1]])
        vec3 = np.zeros(3) if np.linalg.norm(vectors[lst[i][2]]) < 0.000001 else vectors[lst[i][2]] / np.linalg.norm(vectors[lst[i][2]])
        vec4 = np.zeros(3) if np.linalg.norm(vectors[lst[i][3]]) < 0.000001 else vectors[lst[i][3]] / np.linalg.norm(vectors[lst[i][3]])
        
        tmp = vec1 + vec2 + vec3 + vec4
        tmp = np.array([1.0,0.0,0.0]) if np.linalg.norm(tmp) < 0.000001 else  tmp / np.linalg.norm(tmp)
        fl.write("{p1} {p2} {p3}\n".format(p1=tmp[0], 
                                           p2=tmp[1],  
                                           p3=tmp[2]))

100%|██████████| 9323944/9323944 [07:36<00:00, 20427.26it/s]


In [20]:
heart_size = len(np.where(scalars_array > 0.00001)[0])

In [21]:
csv = np.zeros((heart_size, 9))
idx = 0

for i in tqdm(xrange(len(scalars_array))):
    if scalars_array[i] > 0.00001:
        csv[idx, 0] = i                      # index,    dimensionless
        csv[idx, 1] = x = points_array[i, 0] # x,        cm
        csv[idx, 2] = y = points_array[i, 1] # y,        cm
        csv[idx, 3] = z = points_array[i, 2] # z,        cm
        csv[idx, 4] = 0.392       # g_Ks      dimensionless
        csv[idx, 5] = -50000             # amplitude nA 
        csv[idx, 6] = 3                  # duration  ms
        csv[idx, 7] = 0                  # time      ms
        
        csv[idx, 8] = 1                      # model     categorial(1,2,3)
        idx += 1
    
np.savetxt(CHASTE_CSV, csv)

100%|██████████| 1718676/1718676 [00:02<00:00, 609777.01it/s]


In [23]:
heart_size

31779

In [24]:
!cd chaste; tetgen -YrkCV data > trouble.log

In [25]:
!cd chaste; mv data.1.ele data.ele
!cd chaste; mv data.1.node data.node
!cd chaste; mv data.1.face data.face
!cd chaste; rm data.1.*